In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import io
import ogr
import networkx as nx
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import random as ran
import ast 
import json

import folium

In [12]:
import lineas
estaciones=nx.Graph()                      # Creamos una gráfica 
aristas = json.loads(lineas.jason)  

In [13]:
aristas

{'linea5': {'coordinates': [[-99.1492832, 19.5008499],
   [-99.1449058, 19.4896945],
   [-99.1407001, 19.4790945],
   [-99.1369236, 19.4702742],
   [-99.1307974, 19.4634058],
   [-99.1193229, 19.458798],
   [-99.1139048, 19.4580545],
   [-99.105413, 19.4514081],
   [-99.0963578, 19.4511957],
   [-99.0871739, 19.4457529],
   [-99.0876889, 19.4338247],
   [-99.0874207, 19.4240913],
   [-99.0722072, 19.4153591]]},
 'linea4': {'coordinates': [[-99.1070652, 19.482574],
   [-99.1080093, 19.4743304],
   [-99.1119307, 19.4647057],
   [-99.1139048, 19.4580545],
   [-99.1161579, 19.4488638],
   [-99.1182554, 19.4389745],
   [-99.1194355, 19.4288468],
   [-99.1205406, 19.4216528],
   [-99.1217637, 19.409125799999998],
   [-99.1216993, 19.4027355]]},
 'linea12': {'coordinates': [[-99.1878051, 19.3761645],
   [-99.1788508205841, 19.373628712725],
   [-99.1712977237001, 19.3720396532352],
   [-99.1649204, 19.3707294],
   [-99.1567512289475, 19.3705634323458],
   [-99.1540513377261, 19.3617505941329]

In [5]:
for key in aristas:
    i = 0
    ar = aristas[key]['coordinates']
    while i < len(ar)-1:
        x = (ar[i][0],ar[i][1])
        y = (ar[i+1][0],ar[i+1][1])
        estaciones.add_edge(x,y)
        i+=1

estaciones_list= list(estaciones)

print(len(estaciones_list))

164


[(-99.1492832, 19.5008499),
 (-99.1449058, 19.4896945),
 (-99.1407001, 19.4790945),
 (-99.1369236, 19.4702742),
 (-99.1307974, 19.4634058),
 (-99.1193229, 19.458798),
 (-99.1139048, 19.4580545),
 (-99.105413, 19.4514081),
 (-99.0963578, 19.4511957),
 (-99.0871739, 19.4457529),
 (-99.0876889, 19.4338247),
 (-99.0874207, 19.4240913),
 (-99.0722072, 19.4153591),
 (-99.1070652, 19.482574),
 (-99.1080093, 19.4743304),
 (-99.1119307, 19.4647057),
 (-99.1161579, 19.4488638),
 (-99.1182554, 19.4389745),
 (-99.1194355, 19.4288468),
 (-99.1205406, 19.4216528),
 (-99.1217637, 19.409125799999998),
 (-99.1216993, 19.4027355),
 (-99.1878051, 19.3761645),
 (-99.1788508205841, 19.373628712725),
 (-99.1712977237001, 19.3720396532352),
 (-99.1649204, 19.3707294),
 (-99.1567512289475, 19.3705634323458),
 (-99.1540513377261, 19.3617505941329),
 (-99.1429317, 19.3619032),
 (-99.1431697611068, 19.3602749158266),
 (-99.1013199, 19.3561791),
 (-99.1055751254594, 19.3558421753309),
 (-99.1074541104213, 19.3381